In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [2]:
def get_level(row: pd.Series) -> int:
    row = np.array(row.notna(), dtype=np.bool)
    return np.argmax(row) + 1 if row.any() else 0

def remove_spaces(value: str) -> str:
    return value.strip().replace('\n\n', '\n')

def remove_beging_number(value: str) -> str:
    return re.sub(r'\S+\s', '', value, count=1)

def fix_fucking_Ms(value: str) -> str:
    return re.sub(r'^M|М(\d+)', r'M\1', value, count=1)

In [3]:
def process_excel(file, sheet_name: str, skiprows: int = 0, usecols: str = None, columns_names: list[str] = None):
    df = pd.read_excel(file, sheet_name=sheet_name, skiprows=skiprows, usecols=usecols)
    df.columns = [col.strip() for col in df.columns]
    if columns_names is not None:
        df = df[columns_names]
        df = df.rename(columns={columns_names[0]: 'category',
                                columns_names[1]: 'level',
                                columns_names[6]: 'task_value',
                                columns_names[7]: 'control_element'})
    df = df.dropna(subset=['control_element'])
    df['category'] = df['category'].ffill().apply(remove_spaces).apply(fix_fucking_Ms)
    df['task_value'] = df['task_value'].apply(remove_spaces).apply(remove_beging_number)
    df['control_element'] = df['control_element'].apply(remove_spaces)
    df['level'] = df[['level', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']].apply(get_level, axis=1)
    df = df.drop(columns=['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'])
    if len(df.loc[df['level'] == 0]) != 0:
        raise Exception(df.loc[df['level'] == 0])
    return df.reset_index(drop=True)

In [66]:
def to_dict(df: pd.DataFrame) -> dict:
    def get_dict(df: pd.DataFrame) -> list[dict]:
        df['answer_type'] = 'checkbox'
        df['answer_label'] = 'Да/Нет'
        df['answer_type_attributes'] = None
        return dict(zip(range(1, len(df) + 1), df.transpose().to_dict().values()))
    df = df.groupby(['category', 'level'], sort=True)[['task_value', 'control_element']].apply(get_dict)
    df = df.reset_index()
    df = df.groupby('category').apply(lambda grp: dict(zip(grp['level'], grp[0]))).to_dict()
    return df

In [67]:
d = {' Менеджмент': {'skiprows': 6, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ. Как достичь (How):', 'Контрольный элемент. Метод контроля']},
     'Подготовка производства': {'skiprows': 5, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ. Как достичь (How):', 'Контрольный элемент. Метод проверки']},
     'SF-m Ручные операции': {'skiprows': 5, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ. Как достичь (How):', 'Контрольный элемент. Метод проверки']},
     'SF-e оборудование': {'skiprows': 5, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ. Как достичь (How):', 'Контрольный элемент']},
     ' Качество': {'skiprows': 5, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ. Как достичь (How):', 'Контрольный элемент / Способ проверки']},
     'ОТиБ': {'skiprows': 5, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ. Как достичь (How):', 'Контрольный элемент. Метод проверки']},
     'Цепочка поставок': {'skiprows': 4, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ. Как достичь (How):', 'Контрольный элемент']},
     'Техническое развитие': {'skiprows': 5, 'usecols': 'C:L', 'columns_names': ['Категория оценки', 'Уровень', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Требование дорожной карты ПС ЖДРМ', 'Контрольный элемент']}}

In [68]:
data = {}
with tqdm(total=len(d), desc='Processing excel file') as pbar:
    for sheet_name, kwargs in d.items():
        pbar.set_description(f"Processing: \"{sheet_name.strip()}\" sheet")
        df = process_excel('C:/Users/FS/Downloads/ВТРЗ_Дорожная_карта_РОСТ_TOS_v2_ЖДРМ (1) (4).xlsx', sheet_name=sheet_name, **kwargs)
        df = to_dict(df)
        data[sheet_name.strip()] = df
        pbar.update(1)
    pbar.set_description('Done')

Processing: "Менеджмент" sheet:   0%|          | 0/8 [00:00<?, ?it/s]C:\Users\FS\AppData\Local\Temp\ipykernel_11968\3130711547.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('category').apply(lambda grp: dict(zip(grp['level'], grp[0]))).to_dict()
Processing: "Подготовка производства" sheet:  12%|█▎        | 1/8 [00:00<00:04,  1.71it/s]C:\Users\FS\AppData\Local\Temp\ipykernel_11968\3130711547.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the 

In [70]:
import json

In [73]:
with open('result.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)